# **INSHORTS** (https://inshorts.me/)

In [1]:
from tqdm import tqdm
import requests


def get_data(url):
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

    else:
        print("Failed to fetch data. Status code:", response.status_code)
        data = dict()
    
    return data


def get_data_with_count_trial(url):
    counts = [100, 500] + [(i+1) * 1000 for i in range(10)]    
    last_result = None
    
    for idx, count in tqdm(enumerate(counts)):
        transit_url = url.format(count=count)
        response = requests.get(transit_url)

        if response.status_code == 200:
            last_result = response.json()
        
        else:
            break
        
    return last_result["data"]["articles"]

print_ = lambda x: print(x, "news")

In [2]:
all_news_url = "https://inshorts.me/news/all?offset=0&limit={count}"
top_news_url = "https://inshorts.me/news/top?offset=0&limit={count}"
trending_news_url = "https://inshorts.me/news/trending?offset=0&limit={count}"

In [3]:
from datetime import datetime

def timestamp_to_date(timestamp_ms):
    timestamp_seconds = timestamp_ms / 1000  # Convert milliseconds to seconds

    # Convert timestamp to datetime object
    date_object = datetime.fromtimestamp(timestamp_seconds)

    # Format the datetime object as mm-dd-yy
    formatted_date = date_object.strftime('%m-%d-%y')

    return formatted_date


dates = set()
for url in [all_news_url, top_news_url, trending_news_url]:
    timestamp = get_data(url.format(count=1))["data"]["articles"][0]["createdAt"]
    date = timestamp_to_date(timestamp)
    dates.add(date)

    
print(f"{len(dates)} unique dates")
date = dates.pop()
date

2 unique dates


'09-09-23'

# all news

In [4]:
all_news = get_data_with_count_trial(all_news_url)
print_(len(all_news))

5it [00:16,  3.28s/it]

2985 news


# top news

In [5]:
top_news = get_data_with_count_trial(top_news_url)
print_(len(top_news))

6it [00:19,  3.29s/it]

3954 news


# trending news

In [6]:
trending_news = get_data_with_count_trial(trending_news_url)
print_(len(trending_news))

12it [00:07,  1.69it/s]

193 news


# news by topic

In [7]:
# the below 2 are to be used together
get_all_topics = get_data("https://inshorts.me/news/topics")
topic_news_api = lambda topic: get_data(f"https://inshorts.me/news/topics/{topic}")["data"]["articles"]

# get topic names
all_topics = [topic["topic"] for topic in get_all_topics["data"]["topics"]]

# topic news
topic_news = []
for topic in tqdm(all_topics):
    topic_news.extend(topic_news_api(topic))
    
print_(len(topic_news))

100%|███████████████████████████████████████████████████████████████| 18/18 [00:09<00:00,  1.94it/s]

180 news


# news by query

In [8]:
search_news_api = lambda query: get_data_with_count_trial("https://inshorts.me/news/search?query={query}&offset=0&limit={{count}}".format(query=query))

# topic news
search_news = []

for topic in tqdm(all_topics):
    search_news.extend(search_news_api(query=topic))
    
print_(len(search_news))

  0%|                                                                        | 0/18 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:01,  1.60s/it]
2it [00:17,  8.50s/it]
  6%|███▌                                                            | 1/18 [00:17<04:49, 17.01s/it]
0it [00:00, ?it/s]
1it [00:01,  1.45s/it]
2it [00:06,  3.49s/it]
3it [00:16,  5.48s/it]
 11%|███████                                                         | 2/18 [00:33<04:26, 16.69s/it]
0it [00:00, ?it/s]
1it [00:00,  2.70it/s]
2it [00:17,  8.70s/it]
 17%|██████████▋                                                     | 3/18 [00:50<04:15, 17.01s/it]
0it [00:00, ?it/s]
1it [00:01,  1.52s/it]
2it [00:06,  3.68s/it]
3it [00:26,  8.68s/it]
 22%|██████████████▏                                                 | 4/18 [01:16<04:48, 20.57s/it]
0it [00:00, ?it/s]
1it [00:00,  2.28it/s]
2it [00:11,  5.51s/it]
 28%|█████████████████▊                                              | 5/18 [01:27<03:42, 17.13s/it]
0it [00:00, ?it/s]
1it 

7580 news


In [9]:
clubbed_overall_news = []


for news in all_news + top_news + trending_news + topic_news + search_news:
    clubbed_overall_news.append(
        {"title": news["title"].strip(),
         "summary": news["content"].strip(),
         "link": news["sourceUrl"],
         "image_link": news["imageUrl"],
         "source": "inshorts"}
    )
    
print_(len(clubbed_overall_news))

14892 news


In [10]:
clubbed_overall_news[0]

{'title': "How is India-Middle East-Europe Corridor different from China's Belt & Road Initiative?",
 'summary': 'The India-Middle East-Europe Economic Corridor will be substantially different from China\'s Belt and Road Initiative, Railways Minister Ashwini Vaishnaw said. Unlike the BRI, where a huge debt burden gets imposed on host nations, the G20 project will bring in revenue and be bankable, Vaishnaw added. "The BRI came with...conditions...[Now] countries can decide on basis of its needs," he added.',
 'link': 'https://www.news18.com/amp/videos/india/railway-minister-on-how-india-middle-east-eu-corridor-differs-from-china-s-bri-g20-summit-news18-8572165.html?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts',
 'image_link': 'https://static.inshorts.com/inshorts/images/v1/variants/jpg/m/2023/09_sep/10_sun/img_1694367018282_376.jpg?',
 'source': 'inshorts'}

# Deduplicate News

In [11]:
print(len({n["link"] for n in clubbed_overall_news}), " unique links")
print(len(clubbed_overall_news), " total links")

9832  unique links
14892  total links


In [12]:
import sys

sys.path.append('../../../')
from saar.utils import deduplicate_list_of_dicts, get_full_news

keys_to_check = ['title', 'summary', 'link']
clubbed_overall_news = deduplicate_list_of_dicts(clubbed_overall_news, keys_to_check)

print_(len(clubbed_overall_news))

9944 news


[nltk_data] Downloading package punkt to /Users/dawn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **RSS FEEDS**

In [13]:
# import rss_feeds
# import feedparser

In [14]:
# feed_urls = list(set(rss_feeds.rss_feeds))
# feed_news = []

# for feed_url in tqdm(feed_urls):
#     feed = feedparser.parse(feed_url)
    
#     # Iterate through the entries in the feed
#     for entry in feed.entries:
#         try:
#             feed_news.append(
#                 {"title": entry.title,
#                  "summary": entry.summary,
#                  "link": entry.link,
#                  "source": "rss feed"}
#             )
#         except:
#             continue
            
# print_(len(feed_news))

100%|██████████| 264/264 [02:52<00:00,  1.53it/s]

3851 news


In [15]:
# feed_news[0]

{'title': 'Families of babies murdered by nurse Lucy Letby vow to continue their search for answers',
 'summary': 'The families of babies murdered by Lucy Letby have vowed to continue their search for answers as questions swirled around what more could have been done to stop her killing spree.',
 'link': 'https://news.sky.com/story/families-of-babies-murdered-by-lucy-letby-vow-to-continue-their-search-for-answers-12942744',
 'source': 'rss feed'}

# **AGGREGATE**

In [13]:
news = clubbed_overall_news #+ feed_news

# Full Text from URL

In [ ]:
"""
get full news
"""
news = [get_full_news(new) for new in tqdm(news)]
news = [new for new in news if new is not None]

  5%|███                                                       | 519/9944 [43:55<2:58:57,  1.14s/it]

In [ ]:
sys.path.append('../../../')
from saar.infer import Infer

In [17]:
from dotenv import load_dotenv
load_dotenv()


"""
run inference
"""
if len(news) > 0:
    # summary adapter, title adapter path
    summary_adapter_path = os.environ["SUMMARY_ADAPTER_PATH"]
    title_adapter_path = os.environ["TITLE_ADAPTER_PATH"]

    logging.info("loading models..")
    infer = Infer(
        summary_adapter_path=summary_adapter_path, title_adapter_path=title_adapter_path
    )

    # batching
    batch = lambda data, batch_size: [data[i:i + batch_size] for i in range(0, len(data), batch_size)]
    
    batch_size = os.environ["BATCH_SIZE"]
    news = batch(news, batch_size=batch_size)
    
    data = []
    
    logging.info("running inference..")
    for news_batch in tqdm(news):
        # generate summary
        news_batch = infer(mode="summary", data=news_batch)

        # generate title
        # NOTE: Title generation needs summary already generated as "generated_summary" key in the news dict
        news_batch = infer(mode="title", data=news_batch)
        data.extend(news_batch)

In [19]:
import json

path = f"../../../data/training/{date}.json"

with open(path, 'w') as json_file:
    json.dump(data, json_file)